# 데이터 엔지니어링 아키텍쳐

데이터가 원천 시스템에서 최종 붅석 단계까지 흐르는 전체적인 청사진을 설계하는 과정으로

단순히 데이터를 옮기는 것을 넘어, 시스템이 커져도 안정적으로 작동하고 오류가 발생했을 때

빠르게 복구할 수 있는 구조를 만드는 것이 핵심.

**ETL(Extract, Transform, Load)**
과거에는 데이터를 먼저 변환한 뒤 저장하는 ETL 방식이 주를 이뤘다.

**ELT(Extract, Load, Transform)**
최근 클라우드 기술의 발전으로 데이터를 먼저 저장하고 필요할 때 변환하는 ELT 방식이 널리 쓰이고 있다.

이 과정에서 가장 중요한 개념인 멱등성(idempotency)은 동일한 작업을 여러 번 수행해도 결과가 항상 같음을 보장하여 데이터 중복이나 누락을 방지하는 안전장치 역할을 합니다.

**학습 가이드 라인**
- **데이터 흐름의 진화 (ETL vs ELT)** 
    - 전통적인 ETL과 현대적인 ELT의 차이점을 분석하고, 각 방식이 어떤 상황(데이터의 양, 처리 속도 등)에 적합한지 알아봅니다.
 
    - ETL: 데이터 추출 > 데이터 가공 > 데이터 적재
 
    - ELT: 데이터 추출 > 데이터 적재 > 데이터 가공
    
- **견고한 시스템 설계 (모듈화 및 확장성)** 
    - 복잡한 파이프라인을 작은 단위로 쪼개어 관리하는 방법과 데이터가 급증해도 시스템이 멈추지 않게 설계하는 전략을 학습합니다.
        
- **데이터의 신뢰성 확보 (멱등성 및 재시도 로직)**  
    - 장애가 발생했을 때 데이터 오염 없이 안전하게 복구하는 구체적인 로직과 멱등성을 보장하기 위한 설계 기법을 다룹니다.

### 멱등성을 보장하는 저장 전략

| **전략**                          | **설명**                                                    | **특징**                                   |
| ------------------------------- | --------------------------------------------------------- | ---------------------------------------- |
| **Overwrite (덮어쓰기)** 🔁         | 특정 기간이나 범위를 정해두고, 기존 데이터를 지운 뒤 새 데이터로 대체합니다.              | 로직이 단순하고 확실하지만, 데이터 양이 많으면 지우는 비용이 큽니다.  |
| **Upsert (Update + Insert)** 🔄 | 데이터의 고유 키(PK)를 확인하여 이미 있으면 수정(Update), 없으면 삽입(Insert)합니다. | 중복을 세밀하게 방지하지만, 키를 비교하는 과정에서 연산량이 많아집니다. |

### **모듈화(Modularity)** 와 **재시도(Retry)** 전략

거대한 파이프라인을 하나의 덩어리로 만들지 않고,
아래처럼 작은 단위(Task)로 쪼개어 설계했다고 가정해 봅시다.

1. **Task A**: 원천 데이터 추출
2. **Task B**: 데이터 포맷 변환 (JSON -> CSV)
3. **Task C**: 비즈니스 로직 적용 (계산)
4. **Task D**: 최종 데이터베이스 로드

데이터 아키텍처 측면에서 더 효율적으로 대응하려면 각 Task가 독립적으로 작동할 수 있도록 결과를

중간 저장하는 설계가 필요한데, 이를 `체크포인트(CheckPointing)` 또는 `스테이징(Staging)`이라고 부릅니다.

파이프라인을 Task 별로 세분화 한 이유는 에러를 포착하고 재시도 로직을 실행하기 위함입니다.

이를 통해 아래와 같은 이점을 얻을 수 있습니다.

**모듈화된 파이프라인의 이점**
- 부분 재실행: Task C가 실패하면, 이미 성공한 A, B는 건드리지 않고 C부터 다시 시작할 수 있습니다.
  
- 리소스 절약: 수 테라바이트의 데이터를 처리하는 Task A를 매번 다시 실행할 필요가 없어 시간과 비용이 크게 절감됩니다.